In [1]:
import pandas as pd

# 1. Load patient list
patient_df = pd.read_excel("CRE_4399list.xlsx")

# 2. Load part of the lab data (limited to first 100,000 rows)
lab_df = pd.read_excel("CRE_812311labonlydata.xlsx", skiprows=1, nrows=100000)

# 3. Rename columns for clarity
lab_df.columns = ["환자번호", "환자명", "성별", "생년월일", "검사시행일", "검사명", "검사결과",
                  "BMI", "기록당시나이", "수축기혈압", "이완기혈압"]

# 4. Define target tests to include
test_map = {
    "Total Cholesterol": "total",
    "LDL": "ldl",
    "Triglyceride": "tg",
    "HDL": "hdl",
    "eGFR": "egfr",
    "Neutrophil": "neut",
    "Lymphocyte": "lymph",
    "CRP": "crp"
}

# 5. Filter lab rows containing any of the test keywords
pattern = "|".join(test_map.keys())
lab_df = lab_df[lab_df["검사명"].str.contains(pattern, case=False, na=False)].copy()

# 6. Normalize test names
for key, value in test_map.items():
    lab_df.loc[lab_df["검사명"].str.contains(key, case=False, na=False), "검사명정리"] = value

# 7. Keep only the most recent test per patient per test type
lab_df["검사시행일"] = pd.to_datetime(lab_df["검사시행일"], errors="coerce")
lab_df = lab_df.sort_values(["환자번호", "검사명정리", "검사시행일"])
recent_tests = lab_df.drop_duplicates(["환자번호", "검사명정리"], keep="last")

# 8. Pivot to wide format
pivot_tests = recent_tests.pivot(index="환자번호", columns="검사명정리", values="검사결과").reset_index()

# 9. Get most recent BMI / 나이 / 혈압 info
vitals = lab_df.dropna(subset=["BMI", "기록당시나이", "수축기혈압", "이완기혈압"])
vitals = vitals.sort_values(["환자번호", "검사시행일"])
vitals = vitals.drop_duplicates(["환자번호"], keep="last")
vitals = vitals[["환자번호", "BMI", "기록당시나이", "수축기혈압", "이완기혈압"]]

# 10. Merge patient list with vitals and lab results
merged = patient_df.merge(vitals, on="환자번호", how="left")
merged = merged.merge(pivot_tests, on="환자번호", how="left")

# 11. Save to Excel
merged.to_excel("merged_patient_data.xlsx", index=False)
